### **TFM - 50 Labels Reduced Dataset. (08/02/2025)**

* See
https://discuss.huggingface.co/t/most-efficient-multi-label-classifier/9296/2

* The Artificial Guy - MULTI-LABEL TEXT CLASSIFICATION USING BERT AND PYTORCH
https://www.youtube.com/watch?v=f-86-HcYYi8

* Saurabh Anand - BERT for Multi-Label Classification
https://www.youtube.com/watch?v=JjcxZPNZbUY

* KGP Talkie - 5 - Multi-Label Text Classification Model with DistilBERT and Hugging Face Transformers in PyTorch
https://www.youtube.com/watch?v=ZYc9za75Chk

* Fine Tuning BERT for a Multi-Label Classification Problem on Colab - https://medium.com/@abdurhmanfayad_73788/fine-tuning-bert-for-a-multi-label-classification-problem-on-colab-5ca5b8759f3f

* BERT and DistilBERT Models for NLP - https://medium.com/@kumari01priyanka/bert-and-distilbert-model-for-nlp-7352eb16915e

* Choosing the Right Colab Runtime: A Guide for Data Scientists and Analysts - https://drlee.io/choosing-the-right-colab-runtime-a-guide-for-data-scientists-and-analysts-57ee7b7c9638

* distilbert / distilbert-base-uncased - https://huggingface.co/distilbert/distilbert-base-uncased

* "DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter" - https://arxiv.org/abs/1910.01108

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/TFM-MUECIM/*.py /content
!cp /content/drive/MyDrive/TFM-MUECIM/*.txt /content
!cp /content/drive/MyDrive/TFM-MUECIM/*.dat /content
!cp /content/drive/MyDrive/TFM-MUECIM/*.pt /content
!cp /content/drive/MyDrive/TFM-MUECIM/*.csv /content

#!cp /content/drive/MyDrive/TFM-MUECIM/*.tar /content
#!cd /content; tar xf data.tar data
#!cd /content/drive/MyDrive/TFM-MUECIM

In [ ]:
!pip install transformers

In [ ]:
import sys
baseDir = '/content' #/drive/My Drive/TFM-MUECIM'
sys.path.append(baseDir)

Segons el notebook:
Fine-tuning BERT (and friends) for multi-label text classification.ipynb

https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb#scrollTo=HgpKXDfvKBxn  

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Move the model to the correct device before training.

# ensures reproducibility
torch.manual_seed(0)

In [ ]:
from tfm_EURLEX57KDataset import EURLEX57KDataset
from torch.utils.data import random_split

ds = EURLEX57KDataset(baseDir,'50LabelsEURLEX57KDataFrame.csv')
fullSetSize = ds.__len__()
trainSetSize = int(fullSetSize * 0.8)
valSetSize = int(fullSetSize * 0.1)
testSetSize = fullSetSize - trainSetSize - valSetSize
print(f'Full set size: {fullSetSize}')
print(f'Train set size: {trainSetSize}')
print(f'Validation set size: {valSetSize}')
print(f'Test set size: {testSetSize}')
trainData, valData, testData = random_split(
    ds,
    [trainSetSize, valSetSize, testSetSize]
)

Full set size: 29066
Train set size: 23252
Validation set size: 2906
Test set size: 2908


In [ ]:
from torch.utils.data import DataLoader

# set batch size
batchSize = 10

# create dataloaders. In case we'll use a more classical pipeline approach
trainDataLoader = DataLoader(trainData, batch_size=batchSize, shuffle=True)
valDataLoader = DataLoader(valData, batch_size=batchSize, shuffle=True)
testDataLoader = DataLoader(testData, batch_size=batchSize, shuffle=True)

In [ ]:
def countNonZeroItems(items):
    nonZero = torch.nonzero(items, as_tuple= True)
    return len(nonZero[0])

In [ ]:
# iterate through val batches
for i, batch in enumerate(valDataLoader):
  print(f'Batch {i}: ')
  batchFileNames = batch.get('fileName')
  batchData = batch.get('input_ids')
  batchAttentionMasks = batch.get('attention_mask')
  batchLabels = batch.get('labels')

  for elem in zip(batchFileNames, batchData, batchAttentionMasks, batchLabels):
    print(f'fileName: {elem[0]}')
    print(f'input_ids (5 first elements):\n{elem[1][0:5]}')
    print(f'attention_masks (5 first elements):\n{elem[2][0:5]}')
    print(f'Nonzero labels:{countNonZeroItems(elem[3])}\n')

  break

print('Done!')

Batch 0: 
fileName: data/datasets/EURLEX57K/train/32010R0455.json
input_ids (5 first elements):
tensor([3222, 7816, 1006, 7327, 1007])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:2

fileName: data/datasets/EURLEX57K/train/31993R2932.json
input_ids (5 first elements):
tensor([ 3222,  7816,  1006, 25212,  2278])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:1

fileName: data/datasets/EURLEX57K/train/32014R0018.json
input_ids (5 first elements):
tensor([ 3222, 14972,  7816,  1006,  7327])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:2

fileName: data/datasets/EURLEX57K/train/31999R0831.json
input_ids (5 first elements):
tensor([ 3222,  7816,  1006, 14925,  1007])
attention_masks (5 first elements):
tensor([1, 1, 1, 1, 1])
Nonzero labels:1

fileName: data/datasets/EURLEX57K/train/31997D0516.json
input_ids (5 first elements):
tensor([5989, 1013, 4868, 2575, 1013])
attention_masks (5 first elements):
te

In [ ]:
# bert huggingface pretrained model
import os
from transformers import AutoConfig
from transformers import DistilBertForSequenceClassification
from tfm_50LabelsLabelIndex import LabelIndex

labelIndex = LabelIndex(baseDir)

# Gemini. Define a cache directory for Hugging Face models and ensure it exists.
cache_dir = os.path.join(baseDir, 'tfm_cache')
os.makedirs(cache_dir, exist_ok=True)

# Load the configuration with the cache directory.
config = AutoConfig.from_pretrained(
    'distilbert-base-uncased',
    force_download=True,
    cache_dir=cache_dir,
    num_labels=labelIndex.numLabels,
    problem_type='multi_label_classification',
    id2label=labelIndex.id2label,
    label2id=labelIndex.label2id
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
labelIndex.numLabels

50

In [ ]:
# 0 means pretrained fresh model
# 1, 2, 3 ... n. trained epochs from file
last_epoch_trained = 11

if last_epoch_trained == 1:
  modelFile = '20250208_50L_tfm_model_epoch_1.pt'
if last_epoch_trained == 2:
  modelFile = '20250208_50L_tfm_model_epoch_2.pt'
if last_epoch_trained == 3:
  modelFile = '20250208_50L_tfm_model_epoch_3.pt'
if last_epoch_trained == 4:
  modelFile = '20250208_50L_tfm_model_epoch_4.pt'
if last_epoch_trained == 5:
  modelFile = '20250208_50L_tfm_model_epoch_5.pt'
if last_epoch_trained == 6:
  modelFile = '20250208_50L_tfm_model_epoch_6.pt'
if last_epoch_trained == 7:
  modelFile = '20250208_50L_tfm_model_epoch_7.pt'
if last_epoch_trained == 8:
  modelFile = '20250208_50L_tfm_model_epoch_8.pt'
if last_epoch_trained == 9:
  modelFile = '20250208_50L_tfm_model_epoch_9.pt'
if last_epoch_trained == 10:
  modelFile = '20250208_50L_tfm_model_epoch_10.pt'
if last_epoch_trained == 11:
  modelFile = '20250208_50L_tfm_model_epoch_11.pt'



In [ ]:
# Load the model with the configuration and cache directory.
if last_epoch_trained == 0:
  model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', # Changed to the correct model identifier
    config=config,  # Pass the configuration to the model.
    cache_dir=cache_dir  # Specify the cache directory again.
  )
else:
  model = torch.load(os.path.join(baseDir,modelFile), map_location=torch.device(device))

<ipython-input-13-dd5254b7b697>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(os.path.join(baseDir,modelFile), map_location=torch.device(device))


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:

# forward pass. no training. test case
# item = trainData.__getitem__(0)

# outputs = model(
#    input_ids=item['input_ids'][0:512].unsqueeze(0),
#    attention_mask=item['attention_mask'][0:512].unsqueeze(0),
#    labels=item['labels'].unsqueeze(0))

In [ ]:
# outputs.logits[0]


In [ ]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
# calculate metrics
# import numpy as np
# from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(outputs.logits[0])
# threshold = 0.5
# y_pred = np.zeros(probs.shape)
# y_pred[np.where(probs >= threshold)] = 1
# y_true = item['labels'].cpu().numpy() # Convert y_true to a NumPy array
# f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
# roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
# accuracy = accuracy_score(y_true, y_pred)

# metrics = {'f1': f1_micro_average,
#           'roc_auc': roc_auc,
#           'accuracy': accuracy}


In [ ]:
# metrics  # code test

In [ ]:
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
# https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea

import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import torch
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, ):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_true = labels
    y_pred[np.where(probs >= 0.5)] = 1
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # define dictionary of metrics to return
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

def metricsForTestSet():
    predictions = trainer.predict(testData)
    preds = predictions.predictions[0] if isinstance(predictions.predictions, tuple) else predictions.predictions
    labels = predictions.label_ids
    testMetrics = multi_label_metrics(predictions=preds, labels=labels)
    print(testMetrics)

# metric
metricName = 'f1'

# training arguments
trainArgs = TrainingArguments(
    'tfm_oputput',
    report_to='none',  # deactivate wandb  reports. Alternative -> TensorBoard
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batchSize,
    per_device_eval_batch_size=batchSize,
    num_train_epochs=1, # 3 epochs
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metricName)

trainer = Trainer(
    model=model,
    args=trainArgs,
    train_dataset=trainData,
    eval_dataset=valData,
    compute_metrics = compute_metrics,
    #data_collator = Data_Processing(),
)

# API-KEY-WAND-LIBRARY: 1bb618394e7e44feab7f79534fa2be428243d1bb

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Move the model to the correct device before training.
model.to(device)

# epoch 0 - baseline
print(f'Begin epoch {last_epoch_trained + 1} train session - trainer evaluate')
trainer.evaluate()

print('Metrics for test set (before train)')
metricsForTestSet()

# training
print('Epoch train.')
trainer.train()
print('Epoch train done.')

print('End epoch train session - trainer evaluate')
trainer.evaluate()

print('Metrics for test set (after train)')
metricsForTestSet()

print('End epoch train session')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Begin epoch 11 train session - trainer evaluate


Metrics for test set (before train)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-54213d37d9e4>", line 79, in <cell line: 0>
    metricsForTestSet()
  File "<ipython-input-15-54213d37d9e4>", line 36, in metricsForTestSet
    predictions = trainer.predict(testData)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 4151, in predict
    output = eval_loop(
             ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 4267, in evaluation_loop
    losses, logits, labels = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 4

TypeError: object of type 'NoneType' has no len()

In [ ]:
# https://stackoverflow.com/questions/42703500/how-do-i-save-a-trained-model-in-pytorch
import shutil
from datetime import datetime

prefixDate = datetime.today().strftime('%Y%m%d')
fileName = f'{prefixDate}_50L_tfm_model_epoch_{last_epoch_trained + 1}.pt'
modelFullPath = os.path.join(baseDir,fileName)
drivePath = '/content/drive/MyDrive/TFM-MUECIM'
destFullPath = os.path.join(drivePath,fileName)
print('Save model after epoch train session')
torch.save(model, modelFullPath)
shutil.copyfile(modelFullPath, destFullPath)



Save model after epoch train session


'/content/drive/MyDrive/TFM-MUECIM/20250210_50L_tfm_model_epoch_8.pt'